# Notebook used in the calculation of the grid percentages

## Analysis on the intersections between grids and admin 3 boundaries


In [ ]:
#%load_ext jupyter_black
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
pd.set_option("display.float_format", lambda x: "%.5f" % x)
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))/"analysis/02_new_model_input/input/"
)

Creating centroids

In [ ]:
# reading in the intersections file
intersections_area = pd.read_csv(
    input_dir / "phl_adm3_grid_intersections_area.csv"
)
adm3_area = pd.read_csv(input_dir / "adm3_area.csv")
intersections_area["Longitude"] = (
    intersections_area["left"] + intersections_area["right"]
) / 2
intersections_area["Latitude"] = (
    intersections_area["top"] + intersections_area["bottom"]
) / 2
intersections_area["lon_symb"] = [
    "W" if x < 0 else "E" for x in intersections_area["Longitude"]
]
intersections_area["lat_symb"] = [
    "S" if y < 0 else "N" for y in intersections_area["Latitude"]
]
intersections_area["Centroid"] = (
    intersections_area["Longitude"].round(1).astype(str)
    + intersections_area["lon_symb"]
    + "_"
    + intersections_area["Latitude"].round(1).astype(str)
    + intersections_area["lat_symb"]
)
# getting unique grid values
grid_area = intersections_area.drop_duplicates(subset=["id"])
# getting range of grid sizes
grid_area["AreainKM_2"].describe()

In [ ]:
# getting range of municipality sizes
adm3_area["AreainKM"].describe()
# histogram of municipality area
plt.hist(adm3_area["AreainKM"], bins=40)
plt.xlabel("Size in Sqaure KM")
plt.ylabel("Count")
plt.title("Municipality Sizes in Square KM")

In [ ]:
# proportion of municipalities smaller than 120 sq KM
len(adm3_area["AreainKM"][(adm3_area["AreainKM"] <= 120)]) / len(
    adm3_area["AreainKM"]
)

Around 52.8% of municipalities are smaller than 120 square kilmetres.

In [ ]:
# number of municipalities in a grid
municip_no = intersections_area.groupby("id")["ADM3_PCODE"].count()
# municip_no.describe()
# histogram of number of municipalities in a grid
plt.hist(municip_no, bins=10)
plt.xlabel("Municipalities")
plt.ylabel("Count")
plt.title("Number of Municipalities in a Grid")

In [ ]:
intersections_area[intersections_area["id"] == 20556]

In [ ]:
# assigning grids to municipalities based on which one has the largest area
max_area_grid_assignment = (
    intersections_area.sort_values("Area_3", ascending=False)
    .groupby("id", as_index=False)
    .first()
)
max_area_grid_assignment
# assignment rate
len(max_area_grid_assignment["ADM3_PCODE"].unique()) / len(
    intersections_area["ADM3_PCODE"].unique()
)

Around 19% of municipalities do not contribute to grids.

In [ ]:
# writing matching to file
max_assign_out = max_area_grid_assignment[
    ["id", "Centroid", "ADM3_PCODE", "ADM3_EN"]
]
max_assign_out.to_csv(input_dir / "Grid_Matching_Adm3_Max.csv", index=False)

In [ ]:
# creating matrix with weights for each grid
intersections_area["Complete Grid"] = [
    1 if (y - x) <= 100 else 0
    for (y, x) in zip(
        intersections_area["Area_2"], intersections_area["Area_3"]
    )
]
intersections_area["Complete Grid"].describe()
intersections_area["Municipality Completeness"] = (
    intersections_area["Area_3"] / intersections_area["Area"]
)
intersections_area["Grid Completeness"] = (
    intersections_area["Area_3"] / intersections_area["Area_2"]
)
intersections_area[intersections_area["id"] == 11028]

In [ ]:
# writing to CSV files
intersections_area[
    ["id", "Centroid", "ADM3_PCODE", "ADM3_EN", "Grid Completeness"]
].to_csv(input_dir / "Adm3_Perc_inGrid_Weight_Matrix.csv", index=False)
intersections_area[
    ["id", "Centroid", "ADM3_PCODE", "ADM3_EN", "Municipality Completeness"]
].to_csv(input_dir / "Grid_Perc_inAdm3_Weight_Matrix.csv", index=False)